#**스마트폰 센서 데이터 기반 모션 분류**
# 단계3 : 단계별 모델링


## 0.미션

단계별로 나눠서 모델링을 수행하고자 합니다.  

* 단계1 : 정적(0), 동적(1) 행동 분류 모델 생성
* 단계2 : 세부 동작에 대한 분류모델 생성
    * 단계1 모델에서 0으로 예측 -> 정적 행동 3가지 분류 모델링
    * 단계1 모델에서 1으로 예측 -> 동적 행동 3가지 분류 모델링 
* 모델 통합
    * 두 단계 모델을 통합하고, 새로운 데이터에 대해서 최종 예측결과와 성능평가가 나오도록 함수로 만들기
* 성능 비교
    * 기본 모델링의 성능과 비교
    * 모든 모델링은 [다양한 알고리즘 + 성능 튜닝]을 수행해야 합니다.


## 1.환경설정

### (1) 라이브러리 불러오기

* 세부 요구사항
    - 기본적으로 필요한 라이브러리를 import 하도록 코드가 작성되어 있습니다.
    - 필요하다고 판단되는 라이브러리를 추가하세요.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# 필요하다고 판단되는 라이브러리를 추가하세요.





### (2) 데이터 불러오기

* 주어진 데이터셋
    * data01_train.csv : 학습 및 검증용

 <br/>  

* 세부 요구사항
    - data01_train.csv 를 불러와 'data' 이름으로 저장합니다.
        - data에서 변수 subject는 삭제합니다.
    - data01_test.csv 를 불러와 'new_data' 이름으로 저장합니다.


In [2]:
!gdown https://drive.google.com/uc?id=14nndqsX4Scjv9m9tZRK4dOPHnFlVNBNE
!gdown https://drive.google.com/uc?id=1FLLDJCG06Sf4OlXXNatfv6UOqVN57Zjg

Downloading...
From: https://drive.google.com/uc?id=14nndqsX4Scjv9m9tZRK4dOPHnFlVNBNE
To: /content/data01_train.csv
100% 38.6M/38.6M [00:00<00:00, 175MB/s]
Downloading...
From: https://drive.google.com/uc?id=1FLLDJCG06Sf4OlXXNatfv6UOqVN57Zjg
To: /content/features.csv
100% 29.2k/29.2k [00:00<00:00, 6.72MB/s]


In [3]:
data = pd.read_csv('data01_train.csv').drop('subject', axis=1)
features = pd.read_csv('features.csv')

## 2.데이터 전처리

* 세부 요구사항
    - Label 추가 : data 에 Activity_dynamic 를 추가합니다. Activity_dynamic은 과제1에서 is_dynamic과 동일한 값입니다.
    - x와 y1, y2로 분할하시오.
        * y1 : Activity
        * y2 : Activity_dynamic
    - train : val = 8 : 2 혹은 7 : 3
    - random_state 옵션을 사용하여 다른 모델과 비교를 위해 성능이 재현되도록 합니다.

In [4]:
def target_morphs(data, target=None, target2=None):
    df = data.copy()
    # target이 지정되지 않았으면 가장 마지막 열로 지정
    if target is None:
        target = data.columns[-1]
    # targets가 존재하지만 리스트가 아니라면 리스트로 변경
    if target2 is not None:
        if not isinstance(target2, list):
            target2 = [target2]
        # target이 targets에 포함되는지 여부를 target에 저장
        df[target] = df[target].apply(lambda x: 0 if x in target2 else 1)
    return df

In [5]:
from sklearn.metrics import classification_report, confusion_matrix

def score_model(data, random_state=None):
    X, y, lb = data_processing(data)
    x_train, x_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=random_state)
    print("="*100)
    y_pred = double_model(x_train, y_train, x_test)
    print(classification_report(y_test['Activity2'], y_pred))
    print('='*100)
    # 썼던 lb가 주어진다면!
    if lb is not None:
        # LabelEncoder가 숫자형 범주로 바꾸는데 사용한 class들을 가져오고
        values = lb.classes_
        # 그걸 다시 LabelEncoder를 이용해 숫자로 바꿔서
        keys = lb.transform(lb.classes_)
        # 재결합해서 번역 Dict 생성
        dictionary = dict(zip(keys, values))
        for _ in range(len(dictionary)):
            # 그래서 Confusion Matrix를 보기 쉽게 출력
            print(dictionary[_], end=', ')
    print()
    print(confusion_matrix(y_test['Activity2'], y_pred))

In [6]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

def data_processing(data, target='Activity'):
    # target2 리스트에 들어있는 Activity는 0, 없으면 1
    df = target_morphs(data, target=target, target2=['STANDING', 'LAYING', 'SITTING'])
    # 레이블 인코더
    lb = LabelEncoder()
    df[target] = lb.fit_transform(df[target])
    # Unused
    # ynum = len(lb.classes_)
    # 위의 is dynamic이 아닌 실제 값. morphs는 Activity를 덮어씌우게 만들어서 이렇게 따로 추가해줘야함.
    real_y = data[target].copy()
    real_y = lb.fit_transform(real_y)
    # Activity2 에 저장
    df[f'{target}2'] = real_y
    target = [target, f'{target}2']
    X = df.drop(target, axis=1)
    y = df[target]
    # lb를 반환하는 이유는 score_model에서
    return X, y, lb
X, y, lb = data_processing(data)
x_train, x_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=2023)

## **3.단계별 모델링**

![](https://github.com/DA4BAM/image/blob/main/step%20by%20step.png?raw=true)

### (1) 단계1 : 정적/동적 행동 분류 모델

* 세부 요구사항
    * 정적 행동(Laying, Sitting, Standing)과 동적 행동(동적 : Walking, Walking-Up, Walking-Down)을 구분하는 모델 생성.
    * 몇가지 모델을 만들고 가장 성능이 좋은 모델을 선정하시오.

#### 1) 알고리즘1 : 

In [7]:
from sklearn.ensemble import RandomForestClassifier
model1 = RandomForestClassifier()
model1.fit(x_train, y_train.drop('Activity2', axis=1))
y1_pred = model1.predict(x_train)
x_train_a = x_train.iloc[y1_pred == 0]
x_train_b = x_train.iloc[y1_pred == 1]

<ipython-input-7-52e72bd3006f>:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model1.fit(x_train, y_train.drop('Activity2', axis=1))


In [8]:
x_train[:10].index, x_train_a[:10].index, x_train_b[:10].index, y1_pred[:10]

(Int64Index([1313, 5747, 5262, 1949, 1780, 3296, 2035, 5072, 455, 5715], dtype='int64'),
 Int64Index([5747, 5262, 1780, 3296, 2035, 5715, 2418, 398, 3103, 5455], dtype='int64'),
 Int64Index([1313, 1949, 5072, 455, 3254, 1343, 2145, 2521, 4339, 4329], dtype='int64'),
 array([1, 0, 0, 1, 0, 0, 0, 1, 1, 0]))

In [9]:
model2 = RandomForestClassifier()
model2.fit(x_train_a, y_train.drop('Activity', axis=1).iloc[y1_pred == 0])

<ipython-input-9-2ec187c61a67>:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model2.fit(x_train_a, y_train.drop('Activity', axis=1).iloc[y1_pred == 0])


RandomForestClassifier()

In [10]:
model3 = RandomForestClassifier()
model3.fit(x_train_b, y_train.drop('Activity', axis=1).iloc[y1_pred == 1])

<ipython-input-10-751b08850619>:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model3.fit(x_train_b, y_train.drop('Activity', axis=1).iloc[y1_pred == 1])


RandomForestClassifier()

In [11]:
def double_model(x_train, y_train, data):
    # 1차 분류
    model1 = RandomForestClassifier()
    model1.fit(x_train, y_train.drop('Activity2', axis=1))
    # 분류 예측 결과
    y1_pred = model1.predict(x_train)
    # 0이면 여기에
    x_train_a = x_train.iloc[y1_pred == 0]
    # 1이면 여기에
    x_train_b = x_train.iloc[y1_pred == 1]
    # 0 -> 0, 1, 2 분류
    model2 = RandomForestClassifier()
    model2.fit(x_train_a, y_train.drop('Activity', axis=1).iloc[y1_pred == 0])
    # 1 -> 3, 4, 5 분류
    model3 = RandomForestClassifier()
    model3.fit(x_train_b, y_train.drop('Activity', axis=1).iloc[y1_pred == 1])

    temp = model1.predict(data)
    # 원본 순서를 저장
    idx = data.index
    # 데이터 분리
    data_a = data.iloc[temp == 0]
    data_b = data.iloc[temp == 1]
    ans_a = pd.DataFrame({'answer':model2.predict(data_a)}, index=data_a.index)
    ans_b = pd.DataFrame({'answer':model3.predict(data_b)}, index=data_b.index)
    # 결과 합치기
    ans = pd.concat([ans_a, ans_b])
    # 들어온 데이터 순서대로 재정렬해서 반환
    return ans.loc[idx]

In [12]:
y_test

,Activity,Activity2
5574,1,3
2078,0,2
1579,0,0
4585,0,1
5384,0,0
...,...,...
3457,0,2
351,0,2
4997,0,0
2387,0,2


In [13]:
score_model(data, 2023)

<ipython-input-11-3044854e3e60>:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model1.fit(x_train, y_train.drop('Activity2', axis=1))
<ipython-input-11-3044854e3e60>:13: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model2.fit(x_train_a, y_train.drop('Activity', axis=1).iloc[y1_pred == 0])
<ipython-input-11-3044854e3e60>:16: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model3.fit(x_train_b, y_train.drop('Activity', axis=1).iloc[y1_pred == 1])


              precision    recall  f1-score   support

           0       1.00      1.00      1.00       230
           1       0.98      0.94      0.96       209
           2       0.95      0.98      0.96       218
           3       0.99      0.97      0.98       180
           4       0.98      0.98      0.98       165
           5       0.97      0.99      0.98       175

    accuracy                           0.98      1177
   macro avg       0.98      0.98      0.98      1177
weighted avg       0.98      0.98      0.98      1177

LAYING, SITTING, STANDING, WALKING, WALKING_DOWNSTAIRS, WALKING_UPSTAIRS, 
[[229   0   0   0   0   1]
 [  0 197  12   0   0   0]
 [  0   5 213   0   0   0]
 [  0   0   0 175   3   2]
 [  0   0   0   1 162   2]
 [  0   0   0   0   1 174]]


In [15]:
RFmodel = RandomForestClassifier()
RFmodel.fit(x_train, y_train.drop('Activity', axis=1))
RFmodel.score(x_test, y_test.drop('Activity', axis=1))
y_pred = RFmodel.predict(x_test)
print(classification_report(y_test['Activity2'], y_pred))
print(confusion_matrix(y_test['Activity2'], y_pred))

<ipython-input-15-8bd6f9a5be09>:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  RFmodel.fit(x_train, y_train.drop('Activity', axis=1))


              precision    recall  f1-score   support

           0       1.00      1.00      1.00       230
           1       0.98      0.94      0.96       209
           2       0.95      0.98      0.96       218
           3       0.99      0.98      0.98       180
           4       0.98      0.98      0.98       165
           5       0.98      0.99      0.99       175

    accuracy                           0.98      1177
   macro avg       0.98      0.98      0.98      1177
weighted avg       0.98      0.98      0.98      1177

[[230   0   0   0   0   0]
 [  0 197  12   0   0   0]
 [  0   5 213   0   0   0]
 [  0   0   0 176   2   2]
 [  0   0   0   2 161   2]
 [  0   0   0   0   1 174]]


In [ ]:
# To-Do
# 1. Feature importacne 체크
# 2. 하이퍼파라미터 튜닝
# 3. 다른 모델 써보기

In [16]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
import pickle

# 모델 저장
with open('/content/sample_data/random_forest_model.pkl', 'wb') as f:
    pickle.dump(RFmodel, f)

# # 저장된 모델 로드
# with open('/content/sample_data/random_forest_model.pkl', 'rb') as f:
#     loaded_RFmodel = pickle.load(f)

# # 테스트 데이터 예측
# y_pred = loaded_RFmodel.predict(x_test)

# # 모델 평가
# print(classification_report(y_test['Activity2'], y_pred))
# print(confusion_matrix(y_test['Activity2'], y_pred))


#### 2) 알고리즘2 : 

### (2) 단계2-1 : 정적 동작 세부 분류

* 세부 요구사항
    * 정적 행동(Laying, Sitting, Standing)인 데이터 추출
    * Laying, Sitting, Standing 를 분류하는 모델을 생성
    * 몇가지 모델을 만들고 가장 성능이 좋은 모델을 선정하시오.

### (3) 단계2-2 : 동적 동작 세부 분류

* 세부 요구사항
    * 동동적 행동(Walking, Walking Upstairs, Walking Downstairs)인 데이터 추출
    * Walking, Walking Upstairs, Walking Downstairs 를 분류하는 모델을 생성
    * 몇가지 모델을 만들고 가장 성능이 좋은 모델을 선정하시오.

### (4) 분류 모델 합치기


* 세부 요구사항
    * 두 단계 모델을 통합하고, 새로운 데이터(test)에 대해서 최종 예측결과와 성능평가가 나오도록 함수로 만들기
    * 데이터 파이프라인 구축 : test데이터가 로딩되어 전처리 과정을 거치고, 예측 및 성능 평가 수행

![](https://github.com/DA4BAM/image/blob/main/pipeline%20function.png?raw=true)

#### 1) 함수 만들기